In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from core.process_text import process_text
from config import DATA_DIR


[nltk_data] Downloading package punkt to /home/h-pgy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/h-pgy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/h-pgy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/h-pgy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model = joblib.load(os.path.join(DATA_DIR, "svc_model.pkl"))


In [3]:
categories = model.classes_
categories


array(['Assistência Social', 'Cultura',
       'Desenvolvimento Econômico e Trabalho',
       'Direitos Humanos e Cidadania', 'Educação', 'Habitação',
       'Infraestrutura Urbana e Obras', 'Meio Ambiente', 'Outros temas',
       'Saúde', 'Segurança Urbana', 'Transportes e Mobilidade',
       'Urbanismo e Licenciamento',
       'Zeladoria Urbana e melhorias de bairro'], dtype=object)

In [4]:
def predict(frase:str):

    processed_text = process_text(frase)
            

    # Fazer previsão e obter probabilidades
    probabilities = model.decision_function([processed_text])  # Para SVC com 'probability=True', usar predict_proba
    probabilities = np.exp(probabilities) / np.sum(np.exp(probabilities))  # Softmax para converter em probabilidades

    prob_dict = dict(zip(categories, probabilities[0]))

    return prob_dict

In [5]:
preds = predict('Gostaria de mais escolas no meu bairro')
preds

{'Assistência Social': np.float64(0.002842682512236466),
 'Cultura': np.float64(2.1860232253846997e-06),
 'Desenvolvimento Econômico e Trabalho': np.float64(0.03042029509691258),
 'Direitos Humanos e Cidadania': np.float64(1.6391854310990497e-05),
 'Educação': np.float64(0.6437545268386093),
 'Habitação': np.float64(6.1092776697538825e-06),
 'Infraestrutura Urbana e Obras': np.float64(7.944301212389646e-07),
 'Meio Ambiente': np.float64(0.0001276035131897771),
 'Outros temas': np.float64(0.0012956231691581526),
 'Saúde': np.float64(0.009509212430307813),
 'Segurança Urbana': np.float64(0.00012882360776876416),
 'Transportes e Mobilidade': np.float64(0.08278907650080577),
 'Urbanismo e Licenciamento': np.float64(0.00013050835522964617),
 'Zeladoria Urbana e melhorias de bairro': np.float64(0.22897616639045423)}

In [6]:
max_category = max(preds, key=preds.get)
max_prob = preds[max_category]
max_category, max_prob

('Educação', np.float64(0.6437545268386093))

In [7]:
df = pd.read_csv(os.path.join(DATA_DIR, 'dados_final.csv'), sep=';')

In [8]:
df.head()

,audiencia,file_name,duration_audiencia,frase,confidence,channel,offset,duration
0,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,boa noite a todas e todas sou um homem branco ...,0.802756,1,40,27160
1,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,que é a maior cidade do brasil que tem mais re...,0.820823,1,27560,27880
2,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,compostagem mas também pode gerar biogás pros ...,0.894802,1,55840,26120
3,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,fala que tem que ser a a contratação dos das c...,0.819671,0,82400,29040
4,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,é eu tô em são paulo mas se eu atravessar o ou...,0.823547,1,112280,29110


In [9]:
df['prediction_dict'] = df['frase'].apply(predict)

In [10]:
df['best_predict_cat'] = df['prediction_dict'].apply(lambda x: max(x, key=x.get))

In [11]:
df['best_predict_cat'].head()

0    Zeladoria Urbana e melhorias de bairro
1    Zeladoria Urbana e melhorias de bairro
2    Zeladoria Urbana e melhorias de bairro
3    Zeladoria Urbana e melhorias de bairro
4                              Outros temas
Name: best_predict_cat, dtype: object

In [12]:
df['best_predict_proba'] = df.apply(lambda row: row['prediction_dict'][row['best_predict_cat']], axis=1)

In [13]:
df.head()

,audiencia,file_name,duration_audiencia,frase,confidence,channel,offset,duration,prediction_dict,best_predict_cat,best_predict_proba
0,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,boa noite a todas e todas sou um homem branco ...,0.802756,1,40,27160,"{'Assistência Social': 0.0026577765715708897, ...",Zeladoria Urbana e melhorias de bairro,0.639805
1,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,que é a maior cidade do brasil que tem mais re...,0.820823,1,27560,27880,"{'Assistência Social': 0.00324328109074648, 'C...",Zeladoria Urbana e melhorias de bairro,0.639347
2,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,compostagem mas também pode gerar biogás pros ...,0.894802,1,55840,26120,"{'Assistência Social': 0.03028957705192764, 'C...",Zeladoria Urbana e melhorias de bairro,0.639947
3,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,fala que tem que ser a a contratação dos das c...,0.819671,0,82400,29040,"{'Assistência Social': 0.08309286124138808, 'C...",Zeladoria Urbana e melhorias de bairro,0.638506
4,audiencia_publica_geral,audiencia_publica_geral_clip.wav,67.383167,é eu tô em são paulo mas se eu atravessar o ou...,0.823547,1,112280,29110,"{'Assistência Social': 0.0032084880311539173, ...",Outros temas,0.640900


In [16]:
final_f_path = os.path.join(DATA_DIR, 'final_data.csv')
df.to_csv(final_f_path, sep=';', index=False)